In [1]:
path='C:/Users/student/Downloads/'

In [2]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.optimizers import *
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

from nltk.tokenize import sent_tokenize
from konlpy.corpus import kolaw
from konlpy.tag import Okt

Using TensorFlow backend.


In [3]:
chatbotData=pd.read_csv(path+'ChatData.csv')
question, answer = list(chatbotData['Q']), list(chatbotData['A'])

In [4]:
question = question[:100]
answer = answer[:100]

In [6]:
for i in range(10):
    print("질문:" + question[i])
    print("답변:" + answer[i])
    print("\n")

질문:12시 땡!
답변:하루가 또 가네요.


질문:1지망 학교 떨어졌어
답변:위로해 드립니다.


질문:3박4일 놀러가고 싶다
답변:여행은 언제나 좋죠.


질문:3박4일 정도 놀러가고 싶다
답변:여행은 언제나 좋죠.


질문:PPL 심하네
답변:눈살이 찌푸려지죠.


질문:SD카드 망가졌어
답변:다시 새로 사는 게 마음 편해요.


질문:SD카드 안돼
답변:다시 새로 사는 게 마음 편해요.


질문:SNS 맞팔 왜 안하지ㅠㅠ
답변:잘 모르고 있을 수도 있어요.


질문:SNS 시간낭비인 거 아는데 매일 하는 중
답변:시간을 정하고 해보세요.


질문:SNS 시간낭비인데 자꾸 보게됨
답변:시간을 정하고 해보세요.




In [9]:
PAD = "<PADDING>" # 패딩
STA = "<START>" # 시작
END = "<END>" # 끝
OOV = "<OOV>" # out of vocabulary

PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

ENCODER_INPUT = 0
DECODER_INPUT = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 갯수
maxSequences = 30

# 임베딩 벡터 차원
embeddingDim = 100

# LSTM 출력 차원
lstmHiddenDim = 128

# 정규표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [12]:
def posTag(sentences):
    
    tagger = Okt()
    
    sentencePos=[]
    for sentence in sentences:
        # 특수문자 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        sentence = " ".join(tagger.morphs(sentence))
        sentencePos.append(sentence)
    
    return sentencePos

In [13]:
question = posTag(question)
answer = posTag(answer)

In [14]:
question

['12시 땡',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지',
 '가족 있어',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까',
 '감 말랭이 먹고 싶다',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감정 컨트롤 을 못 하겠어',
 '감정 컨

In [21]:
# 질문 + 대답 문장을 하나로 합치기
sentences=[]
sentences.extend(question)
sentences.extend(answer)
len(sentences)

200

In [37]:
# 단어 배열 생성
words=[]
for sentence in sentences:
    for word in sentence.split():
        words.append(word)
        
len(words)

966

In [38]:
# words에서 길이가 0인 단어를 삭제
words = [word for word in words if len(word)>0]

# 중복단어 삭제
words = list(set(words))
len(words)

450

In [39]:
words[:0] = [PAD, STA, END, OOV]
words[:20]

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '하는',
 '볼까',
 '나쁜',
 '있을까',
 '를',
 '좋겠다',
 '말',
 '싶다',
 '난다',
 '반',
 '괜찮아요',
 '취미',
 '적',
 '부모님',
 '됨',
 '온']

In [41]:
# 단어에 대한 인덱스를 부여 -> 딕셔너리
wordToIndex = {word:index for index, word in enumerate(words)}
indexToWord = {index:word for index, word in enumerate(words)}
indexToWord

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '하는',
 5: '볼까',
 6: '나쁜',
 7: '있을까',
 8: '를',
 9: '좋겠다',
 10: '말',
 11: '싶다',
 12: '난다',
 13: '반',
 14: '괜찮아요',
 15: '취미',
 16: '적',
 17: '부모님',
 18: '됨',
 19: '온',
 20: '살찐',
 21: '뭐',
 22: '해보세요',
 23: '당황',
 24: '콕',
 25: '붙잡고',
 26: '옷',
 27: '같은',
 28: '생각',
 29: '맛있게',
 30: '막',
 31: '사이',
 32: '돈',
 33: '장난',
 34: '좋아해주세요',
 35: '자리',
 36: '키울까',
 37: '먹어야지',
 38: '키워',
 39: '켜고',
 40: '저',
 41: '설움',
 42: '했길',
 43: '야',
 44: '가네요',
 45: '중요한',
 46: '마음',
 47: '확실한',
 48: '스트레스',
 49: '가출',
 50: '이야기',
 51: '들더라',
 52: '처럼',
 53: '그',
 54: '선생님',
 55: '싫어',
 56: '돼',
 57: '사는',
 58: '알아차리지',
 59: '옴',
 60: '하루',
 61: '알아차려도',
 62: '쫄딱',
 63: '1',
 64: '불',
 65: '많이',
 66: '건',
 67: '즐거운',
 68: '질질',
 69: '더',
 70: '벗어나는',
 71: '쉬는',
 72: '좋죠',
 73: '걔',
 74: '떨리는',
 75: '기름',
 76: '싶어',
 77: '업무',
 78: '기관',
 79: '지망',
 80: '쇼핑',
 81: '말랭이',
 82: '하고',
 83: '왔나',
 84: '나갔어',
 85: '땡',
 86: '보고',
 87: '가까워질',
 88: '오늘

In [51]:
# 문장 -> 인덱스로 변환
def convertTextToIndex(sentences, voc, mytype):
    
    sentencesIndex=[]
    for sentence in sentences:
        
        sentenceIndex=[]
        if mytype == DECODER_INPUT:
            
            sentenceIndex.extend([voc[STA]])
            
        for word in sentence.split():
            
            if voc.get(word) is not None: # 단어에 해당하는 인덱스가 있는 경우
                sentenceIndex.extend([voc[word]]) # 단어에 해당되는 인덱스가 추가
                
            else: # 사전에 없는 단어의 경우 OOV추가
                sentenceIndex.extend([voc[OOV]])
                
        if mytype == DECODER_TARGET:
            
            # 디코더 출력은 맨 마지막에 end 추가
            if maxSequences <= len(sentenceIndex):
                sentenceIndex = sentenceIndex[:maxSequences-1] + [voc[END]]
                
            else:
                sentenceIndex += [voc[END]]
                
        else:
            if len(sentenceIndex) > maxSequences:
                sentenceIndex = sentenceIndex[:maxSequences]
                
        # 0으로 채움(pad_sequence)
        sentenceIndex += [wordToIndex[PAD]] * (maxSequences-len(sentenceIndex))
        
        sentencesIndex.append(sentenceIndex)
            
    return np.asarray(sentencesIndex)

In [52]:
# 인코더 입력, 디코더 입력, 디코더 출력 -> 인덱스 변환
xEncoder = convertTextToIndex(question, wordToIndex, ENCODER_INPUT)
print(xEncoder[0])

xDecoder = convertTextToIndex(answer, wordToIndex, DECODER_INPUT)
print(xDecoder[0])

yDecoder = convertTextToIndex(answer, wordToIndex, DECODER_TARGET)
print(yDecoder[0])

[442  85   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[  1  60 291 161  44   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[ 60 291 161  44   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [8]:
# 인코더 입력 : 12시 땡
# 디코더 입력 : START 하루 가 또 가네요 
# 디코더 출력 : 하루 가 또 가네요 END0000000

In [18]:
len(words)

454

In [19]:
#np.zeros((2,3,4))
oneHotData=np.zeros((len(yDecoder),maxSequences,len(words)))
#100(답변개수), 30(최대 단어 개수), 454(전체 단어 집합 개수)

In [20]:
np.shape(oneHotData)#디코더 출력
#하루 가 또 가네요 END
for i, seq in enumerate(yDecoder):# (100,30)
    for j, index in enumerate(seq):
        oneHotData[i,j,index]=1
yDecoder=oneHotData

In [21]:
yDecoder[0].shape #첫번째 답변 내용의 shape

(30, 454)

In [22]:
#훈련 모델 생성
#인코더 정의

#입력 문장의 인덱스 sequence를 입력
encoderInputs=layers.Input(shape=(None,))
#임베딩 계층
encoderOutputs=layers.Embedding(len(words),embeddingDim)(encoderInputs)

encoderOutputs,stateH, stateC=layers.LSTM(lstmHiddenDim,return_state=True, 
            dropout=0.2, recurrent_dropout=0.5)(encoderOutputs)
#return_state=True => 상태값 리턴
#LSTM은 2개 상태 존재(셀, 히든 스테이트)

encoderStates=[stateH, stateC]


#디코더 정의
#출력 문장의 인덱스 sequence를 입력
decoderInputs=layers.Input(shape=(None,))
#임베딩 계층
decoderEmbedding=layers.Embedding(len(words),
                                embeddingDim)
decoderOutputs=decoderEmbedding(decoderInputs)



decoderLSTM=layers.LSTM(lstmHiddenDim,
                        return_state=True, 
            return_sequences=True, 
                        dropout=0.2, 
                        recurrent_dropout=0.5)
decoderOutputs, _, _=decoderLSTM(decoderOutputs,initial_state=encoderStates)
decoderDense=layers.Dense(len(words), 
                          activation="softmax")
decoderOutputs=decoderDense(decoderOutputs)


Instructions for updating:
Colocations handled automatically by placer.


In [23]:
model=models.Model([encoderInputs, decoderInputs],
             decoderOutputs)

In [24]:
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [25]:
#예측 모델 인코더 정의
encoderModel=models.Model(encoderInputs, 
                          encoderStates)

#예측 모델 디코더 정의
#바로 앞에 있는 디코더의 출력(상태)을 입력 받아서
#예측을 해야 함.
decoderStateInputH=layers.Input(shape=(lstmHiddenDim,))
decoderStateInputC=layers.Input(shape=(lstmHiddenDim,))
decoderStatesInputs=[decoderStateInputH,decoderStateInputC]

#임베딩 계층
decoderOutputs=decoderEmbedding(decoderInputs)
#LSTM 계층
decoderOutputs, stateH, stateC=decoderLSTM(decoderOutputs,
           initial_state=decoderStatesInputs)
decoderStates=[stateH, stateC]

#Dense계층을 통해 원핫 형식으로 예측 단어 인덱스를 추출
decoderOutputs=decoderDense(decoderOutputs)

#예측 모델 디코더 설정
decoderModel=models.Model([decoderInputs]+decoderStatesInputs,
            [decoderOutputs]+decoderStates)

In [26]:
#인덱스를 문장으로 변환
def convertIndexToText(indexs, voc):
    #구현
    sentence=""
    for i in indexs:
        if i==END_INDEX: #종료 인덱스
            break;
        if voc.get(i) is not None:
            sentence+=voc[i]
        else:
            sentence.extend([voc[OOV_INDEX]])
        sentence +=" "    
    return sentence    

In [27]:
#에폭
for epoch in range(10):
    print("total epoch:", epoch+1)
    history=model.fit([xEncoder,xDecoder], yDecoder,
             epochs=100,
             batch_size=64,
                     verbose=0)
    print("accuracy :", history.history['accuracy'])
    print("loss :", history.history['loss'])
    #문장 예측
    #3박 4일 놀러 가고 싶다 -> 여행 은 언제나 좋죠
    
    inputEncoder=xEncoder[2].reshape(1,xEncoder[2].shape[0]) #(30,) ->(1,30)
    inputDecoder=xDecoder[2].reshape(1,xDecoder[2].shape[0]) #(30,) ->(1,30)
    
    results=model.predict([inputEncoder,inputDecoder])
    
    #결과값에 대해서 가장 큰 값의 위치를 구함
    index=np.argmax(results[0], 1)
    #인덱스 -> 문장으로 변환
    sentence=convertIndexToText(index, indexToWord)
    print(sentence)
    print()
    

total epoch: 1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
accuracy : [0.28433332, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7863333, 0.7866667, 0.78833336, 0.7863333, 0.78966665, 0.789, 0.789, 0.79, 0.78933334, 0.792, 0.7913333, 0.7916667, 0.7926667, 0.795, 0.79066664, 0.79333335, 0.795, 0.7966667, 0.79333335, 0.79433334, 0.79966664, 0.798, 0.798, 0.79833335, 0.798, 0.796, 0.801, 0.80366665, 0.8056667, 0.804, 0.80633336, 0.80466664, 0.807, 0.80766666, 0.8056667, 0.81266665, 0.8103333, 0.8146667, 0.817, 0.81133336, 0.8056667, 0.81366664, 0.816, 0.818, 0.81733334, 0.818, 0.8196667, 0.82, 0.82033336, 0.82133335, 0.8206667, 0.823, 0.8233333, 0.824, 0.823, 0.82266665, 0.824, 0.8246667, 0.8246667, 0.8236667, 0.8243333, 0.8246667, 0.8243333, 0.826, 0.825, 0.82566667, 0.82566667, 0.825, 0.8286667, 0.825, 0.82533336, 0.827, 0.828, 0.

accuracy : [0.955, 0.95633334, 0.9576667, 0.9583333, 0.95666665, 0.9576667, 0.95666665, 0.9573333, 0.95966667, 0.9573333, 0.96, 0.95933336, 0.96033335, 0.959, 0.958, 0.961, 0.96066666, 0.96133333, 0.96033335, 0.96066666, 0.96, 0.96033335, 0.96066666, 0.961, 0.9626667, 0.962, 0.9626667, 0.9633333, 0.9636667, 0.9636667, 0.96433336, 0.9633333, 0.963, 0.9636667, 0.9626667, 0.962, 0.96433336, 0.961, 0.964, 0.961, 0.9636667, 0.96433336, 0.965, 0.96466666, 0.9636667, 0.964, 0.96666664, 0.96533334, 0.96433336, 0.96466666, 0.964, 0.96433336, 0.965, 0.96566665, 0.966, 0.9663333, 0.96566665, 0.967, 0.96566665, 0.9676667, 0.9663333, 0.96566665, 0.9663333, 0.96566665, 0.966, 0.9673333, 0.9663333, 0.96666664, 0.9673333, 0.9673333, 0.966, 0.967, 0.967, 0.96666664, 0.96666664, 0.966, 0.9663333, 0.966, 0.968, 0.969, 0.96666664, 0.96466666, 0.9663333, 0.9686667, 0.968, 0.9673333, 0.9686667, 0.968, 0.967, 0.967, 0.9686667, 0.9673333, 0.968, 0.969, 0.9686667, 0.9683333, 0.97, 0.9676667, 0.97033334, 0.969]

accuracy : [0.9726667, 0.9723333, 0.972, 0.9716667, 0.9716667, 0.9716667, 0.9723333, 0.97333336, 0.9726667, 0.972, 0.9726667, 0.9716667, 0.972, 0.9726667, 0.9713333, 0.9726667, 0.972, 0.9716667, 0.9713333, 0.97066665, 0.972, 0.9726667, 0.9726667, 0.9723333, 0.9726667, 0.97366667, 0.9716667, 0.973, 0.9726667, 0.97, 0.97066665, 0.972, 0.973, 0.9723333, 0.9723333, 0.9713333, 0.971, 0.97433335, 0.9723333, 0.97333336, 0.973, 0.97333336, 0.9726667, 0.97366667, 0.97366667, 0.9723333, 0.9713333, 0.9726667, 0.9716667, 0.973, 0.9723333, 0.9723333, 0.973, 0.9713333, 0.973, 0.973, 0.9726667, 0.971, 0.973, 0.97333336, 0.9713333, 0.97366667, 0.97366667, 0.9713333, 0.97066665, 0.9726667, 0.973, 0.973, 0.9726667, 0.9723333, 0.9723333, 0.973, 0.97366667, 0.972, 0.9726667, 0.97366667, 0.9713333, 0.9726667, 0.974, 0.9723333, 0.97366667, 0.974, 0.974, 0.9723333, 0.9726667, 0.9716667, 0.9723333, 0.972, 0.9726667, 0.9723333, 0.973, 0.97333336, 0.97366667, 0.9723333, 0.9726667, 0.9723333, 0.9716667, 0.972333

accuracy : [0.97433335, 0.973, 0.974, 0.9766667, 0.975, 0.97533333, 0.9766667, 0.97566664, 0.9766667, 0.97533333, 0.97433335, 0.97533333, 0.97466666, 0.97433335, 0.97533333, 0.97533333, 0.97533333, 0.97366667, 0.975, 0.97566664, 0.9766667, 0.97533333, 0.976, 0.9766667, 0.974, 0.97466666, 0.976, 0.97566664, 0.97466666, 0.97533333, 0.975, 0.97566664, 0.97433335, 0.977, 0.977, 0.975, 0.974, 0.97433335, 0.97566664, 0.974, 0.97533333, 0.97433335, 0.97533333, 0.97366667, 0.97466666, 0.976, 0.97566664, 0.97566664, 0.9766667, 0.9763333, 0.97533333, 0.976, 0.9766667, 0.97466666, 0.9763333, 0.97566664, 0.97533333, 0.974, 0.9763333, 0.976, 0.976, 0.97566664, 0.974, 0.976, 0.97533333, 0.97533333, 0.974, 0.97566664, 0.97533333, 0.9763333, 0.97533333, 0.97533333, 0.97566664, 0.975, 0.977, 0.975, 0.9763333, 0.97533333, 0.97566664, 0.97533333, 0.97566664, 0.975, 0.97466666, 0.97466666, 0.976, 0.977, 0.9773333, 0.97433335, 0.97433335, 0.97833335, 0.976, 0.975, 0.97866666, 0.97466666, 0.974, 0.97566664,

In [ ]:
# xEncoder[2].shape #(30,)
# xDecoder[2].shape #(30,)

In [ ]:
#"3박4일 놀러가고 싶다" #여행은 언제나 좋죠
#"3박4일 같이 놀러가고 싶다" #여행은 언제나 좋죠
#"4박5일 놀러가고 싶다"  ???
#"3박4일 동안 동해로 놀러가고 싶다" ???
#"3박4일 동안 동해로 놀러가려고" ???

In [34]:
# 문장 입력
#[[320, 157, ..., 19,0,0,0,0...,0]]
def makePredictInput(sentence):    
    sentences=[]
    sentences.append(sentence)
    sentences=posTag(sentences)
    inputSeq=convertTextToIndex(sentences,
                                wordToIndex,
                                ENCODER_INPUT)
    return inputSeq

In [39]:
np.zeros((1,1))[0,0]

0.0

In [40]:
#예측 답변 생성
def generateText(inputSeq):
        #입력을 인코더에 넣고, 마지막 상태 구함
        states=encoderModel.predict(inputSeq)
        
        #목표 시퀀스 초기화
        targetSeq=np.zeros((1,1))
        #<START> 시그널을 추가
        targetSeq[0,0]=STA_INDEX
        #인덱스 초기화
        indexs=[]
        
        #디코더 반복
        while 1:
            decoderOuputs, stateH, stateC=decoderModel.predict([targetSeq]+states)
            #결과를 원핫인코딩 형식으로 변환
            index=np.argmax(decoderOuputs[0,0,:])
            indexs.append(index)
            #종료 체크
            if index==END_INDEX or len(indexs)>=maxSequences:
                break
                
                #targetSeq를 이전 출력으로 설정
            targetSeq=np.zeros((1,1))
            targetSeq[0,0]=index    #STA_INDEX 
            
            #디코더의 이전 상태를 다음 디코더 예측에 사용
            states=[stateH, stateC]
            
        #인덱스를 문장으로 변환
        sentence=convertIndexToText(indexs, indexToWord)
        
        return sentence                         
        
    
    

In [59]:
#입력문장 -> 인덱스
inputSeq=makePredictInput("놀러가려고")
#[[320, 157, ..., 19,0,0,0,0...,0]]

In [60]:
sentence=generateText(inputSeq)
sentence #여행은 언제나 좋죠

'다음 달 에는 더 절약 해봐요 '